In [1]:
# created by Martti
#setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

gd_train=pd.read_csv("Data/Train_Plant1_Corrected.csv")
gd_test=pd.read_csv("Data/Test_Plant1_Corrected.csv")


#gd=gd_train
#gd=gd_test

In [2]:
#gd_train1=pd.read_csv("Data/Train_Plant2.csv")
#gd_test1=pd.read_csv("Data/Test_Plant2.csv")

In [3]:
#gd_test

# NB! sõltuvalt Train/test switch

In [4]:
gd_train[48576:]

,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,ARTIFICIAL
48576,2020-06-07 00:00:00,1BY6WEcLGh8j5v7,22.055740,20.29718,0.0,4135001.0,0.0,0.0,0.000000,6418482.0,0
48577,2020-06-07 00:00:00,1IF53ai7Xc0U56Y,22.055740,20.29718,0.0,4135001.0,0.0,0.0,0.000000,6355099.0,0
48578,2020-06-07 00:00:00,3PZuoBAID5Wc2HD,22.055740,20.29718,0.0,4135001.0,0.0,0.0,812.000000,7158982.0,0
48579,2020-06-07 00:00:00,7JYdWkrLSPkdwr4,22.055740,20.29718,0.0,4135001.0,0.0,0.0,789.625000,7770457.0,0
48580,2020-06-07 00:00:00,McdE0feGgRqW7Ca,22.055740,20.29718,0.0,4135001.0,0.0,0.0,0.000000,7331160.0,0
...,...,...,...,...,...,...,...,...,...,...,...
54907,2020-06-09 23:45:00,wCURE6d3bPkepu2,22.825907,20.68654,0.0,4135001.0,0.0,0.0,3432.857143,6975980.0,0
54908,2020-06-09 23:45:00,z9Y9gH1T5YWrNuG,22.825907,20.68654,0.0,4135001.0,0.0,0.0,7629.000000,7199053.0,0
54909,2020-06-09 23:45:00,zBIq5rxdHJRwDNY,22.825907,20.68654,0.0,4135001.0,0.0,0.0,8136.000000,6531405.0,0
54910,2020-06-09 23:45:00,zVJPv84UY57bAof,22.825907,20.68654,0.0,4135001.0,0.0,0.0,8289.000000,7310456.0,0


In [5]:
#gd_train[48576:]
#24*4*22*3=6336
#53856-6336= 47520

#alates 2020-06-07

#Test andmestik
gd=pd.concat([gd_train[48576:],gd_test], ignore_index=True)

#Train andmestik
#gd=pd.concat([gd_train,gd_test[:1056]], ignore_index=True)

In [6]:
#Add columns
gd['DC_POWER_DIF_AVG_PER_ALL']=gd['DC_POWER']-gd['DC_POWER'].mean()
gd['AC_POWER_DIF_AVG_PER_ALL']=gd['AC_POWER']-gd['AC_POWER'].mean()
gd['DATE_TIME'] = pd.to_datetime(gd['DATE_TIME'], errors='coerce')
gd['DATE']=pd.to_datetime(gd['DATE_TIME'].dt.date)
gd['TIME']=gd['DATE_TIME'].dt.time

In [7]:
#New columns per inverter 
uurimiseks= gd.copy()
groups = dict(list(uurimiseks.groupby("SOURCE_KEY")))
shift_nr=3
shift_columns = ['DC_POWER','AC_POWER','DAILY_YIELD','TOTAL_YIELD','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION']
gr_nr=0

#inverter based shifting
for group in groups:
    gr_nr+=1
    dates=groups[group]['DATE'].unique()
    for i in range(shift_nr):
        for j in range(len(shift_columns)):
            groups[group][shift_columns[j]+'_PER_INVERTER_MOMENT_SHIFT_MINUS_'+str(i+1)]=groups[group][shift_columns[j]].shift(periods=(i+1), fill_value=0)
            groups[group][shift_columns[j]+'_PER_INVERTER_DAY_SHIFT_MINUS_'+str(i+1)]=groups[group][shift_columns[j]].shift(periods=(i+1)*96, fill_value=0)
    
    #groups[group]['INVERTER']=group - ebaoluline
    groups[group]['DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD'] = groups[group]['DC_POWER'] - np.mean(groups[group]['DC_POWER']) 
    groups[group]['AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD'] = groups[group]['AC_POWER'] - np.mean(groups[group]['AC_POWER']) 
    
    ############ per one day comparison, but note sure if correct
    gr_no2=0
    for date in dates:
        gr_no2 +=1
        d=groups[group][groups[group]['DATE']==date]
        d['DC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['DC_POWER'] - np.mean(d['DC_POWER'])
        d['AC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['AC_POWER'] - np.mean(d['AC_POWER'])
        if gr_no2==1:
            date_data=pd.DataFrame(columns = d.keys())
            
        date_data =pd.concat([date_data, d], ignore_index=True)
    #########
    
    if gr_nr==1:
        gen_data=pd.DataFrame(columns = date_data.keys())
    
    #gen_data.append(groups[group], ignore_index=True) - did not seem to work and was replaced with concat
    gen_data=pd.concat([gen_data,date_data], ignore_index=True)
    
#gen_data=gen_data.fillna(value=0) - ebaoluline
gen_data

<ipython-input-7-e7170d9f046c>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['DC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['DC_POWER'] - np.mean(d['DC_POWER'])
<ipython-input-7-e7170d9f046c>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['AC_DIF_AVG_PER_INVERTER_PER_DAY'] = d['AC_POWER'] - np.mean(d['AC_POWER'])


,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,...,AMBIENT_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_3,AMBIENT_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_3,MODULE_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_3,MODULE_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_DAY_SHIFT_MINUS_3,DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,DC_DIF_AVG_PER_INVERTER_PER_DAY,AC_DIF_AVG_PER_INVERTER_PER_DAY
0,2020-06-07 00:00:00,1BY6WEcLGh8j5v7,22.055740,20.297180,0.0,4135001.0,0.0,0.0,0.0,6418482.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-2578.165755,-252.302118,-2262.684524,-221.328925
1,2020-06-07 00:15:00,1BY6WEcLGh8j5v7,22.103192,20.396224,0.0,4135001.0,0.0,0.0,0.0,6418482.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-2578.165755,-252.302118,-2262.684524,-221.328925
2,2020-06-07 00:30:00,1BY6WEcLGh8j5v7,22.224561,20.518799,0.0,4135001.0,0.0,0.0,0.0,6418482.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-2578.165755,-252.302118,-2262.684524,-221.328925
3,2020-06-07 00:45:00,1BY6WEcLGh8j5v7,22.306429,20.727631,0.0,4135001.0,0.0,0.0,0.0,6418482.0,...,22.055740,0.000000,20.297180,0.000000,0.0,0.0,-2578.165755,-252.302118,-2262.684524,-221.328925
4,2020-06-07 01:00:00,1BY6WEcLGh8j5v7,22.263181,20.769368,0.0,4135001.0,0.0,0.0,0.0,6418482.0,...,22.103192,0.000000,20.396224,0.000000,0.0,0.0,-2578.165755,-252.302118,-2262.684524,-221.328925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23227,2020-06-17 22:45:00,zVJPv84UY57bAof,22.150570,21.480377,0.0,4135001.0,0.0,0.0,5910.0,7363272.0,...,22.563179,23.522461,21.164713,21.420634,0.0,0.0,-3011.039109,-294.515462,-2530.472545,-247.621276
23228,2020-06-17 23:00:00,zVJPv84UY57bAof,22.129816,21.389024,0.0,4135001.0,0.0,0.0,5910.0,7363272.0,...,22.199229,23.386562,20.515273,21.151996,0.0,0.0,-3011.039109,-294.515462,-2530.472545,-247.621276
23229,2020-06-17 23:15:00,zVJPv84UY57bAof,22.008275,20.709211,0.0,4135001.0,0.0,0.0,5910.0,7363272.0,...,22.171737,23.331081,21.080829,21.470425,0.0,0.0,-3011.039109,-294.515462,-2530.472545,-247.621276
23230,2020-06-17 23:30:00,zVJPv84UY57bAof,21.969495,20.734963,0.0,4135001.0,0.0,0.0,5910.0,7363272.0,...,22.150570,23.444953,21.480377,22.455490,0.0,0.0,-3011.039109,-294.515462,-2530.472545,-247.621276


# Validation by comparison

In [8]:
#gen_data['AC_POWER'][20000:20004]

In [9]:
#gen_data['AC_POWER_PER_INVERTER_MOMENT_SHIFT_MINUS_1'][20000:20004]

In [10]:
#a=gen_data[gen_data['SOURCE_KEY']=='1BY6WEcLGh8j5v7']
#b=pd.concat([a[a['DATE_TIME']=='2020-05-17 13:00:00'],a[a['DATE_TIME']=='2020-05-16 13:00:00']], ignore_index=True)
#b['AC_POWER_PER_INVERTER_DAY_SHIFT_MINUS_1']

In [11]:
#b['AC_POWER']

In [12]:
#groups[group].keys()

# Other fields

In [13]:
#Group by day labels
'''
* label "FAULTY_according_to_DAILY_YIELD" - yield stays the same, while other inverters still increases
* label "INCORRECT_daily_yield" - yield per day decreases
* columns per inverter day average differences (total are done)
'''

uurimiseks= gen_data.copy()
groups = dict(list(uurimiseks.groupby("DATE")))
shift_nr=3
production_columns = ['DC_POWER','AC_POWER','DAILY_YIELD','TOTAL_YIELD', 'DC_AVG_DIF_PER_DAY','AC_AVG_DIF_PER_DAY']
gr_nr=0

for group in groups:
    groups[group]['DC_AVG_DIF_PER_DAY'] =  groups[group]['DC_POWER'] -np.mean(groups[group]['DC_POWER']) 
    groups[group]['AC_AVG_DIF_PER_DAY'] =  groups[group]['AC_POWER'] - np.mean(groups[group]['AC_POWER']) 
    
    gr_nr+=1
    '''
    Tundub et toodaks vale infot
    for i in range(shift_nr):
        for j in range(len(production_columns)):
            groups[group][production_columns[j]+'_PER_DAY_SHIFT_'+str(i+1)]=groups[group][production_columns[j]].shift(periods=-(i+1), fill_value=0)
    '''
    if gr_nr==1:
        gen_data2=pd.DataFrame(columns =groups[group].keys())
    
    gen_data2=pd.concat([gen_data2,groups[group]], ignore_index=True)
    
gen_data2

,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,...,MODULE_TEMPERATURE_PER_INVERTER_MOMENT_SHIFT_MINUS_3,MODULE_TEMPERATURE_PER_INVERTER_DAY_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_DAY_SHIFT_MINUS_3,DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,DC_DIF_AVG_PER_INVERTER_PER_DAY,AC_DIF_AVG_PER_INVERTER_PER_DAY,DC_AVG_DIF_PER_DAY,AC_AVG_DIF_PER_DAY
0,2020-06-07 00:00:00,1BY6WEcLGh8j5v7,22.055740,20.297180,0.0,4135001.0,0.0,0.0,0.0,6418482.0,...,0.000000,0.000000,0.0,0.0,-2578.165755,-252.302118,-2262.684524,-221.328925,-3255.892122,-318.247867
1,2020-06-07 00:15:00,1BY6WEcLGh8j5v7,22.103192,20.396224,0.0,4135001.0,0.0,0.0,0.0,6418482.0,...,0.000000,0.000000,0.0,0.0,-2578.165755,-252.302118,-2262.684524,-221.328925,-3255.892122,-318.247867
2,2020-06-07 00:30:00,1BY6WEcLGh8j5v7,22.224561,20.518799,0.0,4135001.0,0.0,0.0,0.0,6418482.0,...,0.000000,0.000000,0.0,0.0,-2578.165755,-252.302118,-2262.684524,-221.328925,-3255.892122,-318.247867
3,2020-06-07 00:45:00,1BY6WEcLGh8j5v7,22.306429,20.727631,0.0,4135001.0,0.0,0.0,0.0,6418482.0,...,20.297180,0.000000,0.0,0.0,-2578.165755,-252.302118,-2262.684524,-221.328925,-3255.892122,-318.247867
4,2020-06-07 01:00:00,1BY6WEcLGh8j5v7,22.263181,20.769368,0.0,4135001.0,0.0,0.0,0.0,6418482.0,...,20.396224,0.000000,0.0,0.0,-2578.165755,-252.302118,-2262.684524,-221.328925,-3255.892122,-318.247867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23227,2020-06-17 22:45:00,zVJPv84UY57bAof,22.150570,21.480377,0.0,4135001.0,0.0,0.0,5910.0,7363272.0,...,21.164713,21.420634,0.0,0.0,-3011.039109,-294.515462,-2530.472545,-247.621276,-2517.888173,-246.393720
23228,2020-06-17 23:00:00,zVJPv84UY57bAof,22.129816,21.389024,0.0,4135001.0,0.0,0.0,5910.0,7363272.0,...,20.515273,21.151996,0.0,0.0,-3011.039109,-294.515462,-2530.472545,-247.621276,-2517.888173,-246.393720
23229,2020-06-17 23:15:00,zVJPv84UY57bAof,22.008275,20.709211,0.0,4135001.0,0.0,0.0,5910.0,7363272.0,...,21.080829,21.470425,0.0,0.0,-3011.039109,-294.515462,-2530.472545,-247.621276,-2517.888173,-246.393720
23230,2020-06-17 23:30:00,zVJPv84UY57bAof,21.969495,20.734963,0.0,4135001.0,0.0,0.0,5910.0,7363272.0,...,21.480377,22.455490,0.0,0.0,-3011.039109,-294.515462,-2530.472545,-247.621276,-2517.888173,-246.393720


In [14]:
#groups[group].keys()

In [15]:
# add inverter difference from average per that moment
'''
* columns "MOMENT_AVERAGE" - average value per this moment of inverter production info
* columns "DIF_FROM_AVERAGE" - inverter avg values in comparison with other inverters per this moment
'''

uurimiseks= gen_data2.copy()
groups = dict(list(uurimiseks.groupby("DATE_TIME")))
shift_nr=3
production_columns = ['DC_POWER','AC_POWER','DAILY_YIELD','TOTAL_YIELD', 'DC_DIF_AVG_PER_DAY','AC_DIF_AVG_PER_DAY']
gr_nr=0

for group in groups:
    groups[group]['DC_DIF_AVG_PER_MOMENT'] =  groups[group]['DC_POWER'] - np.mean(groups[group]['DC_POWER']) 
    groups[group]['AC_DIF_AVG_PER_MOMENT'] =  groups[group]['AC_POWER'] -np.mean(groups[group]['AC_POWER']) 

    gr_nr+=1
    ''' 
    tundub valet tulemust andvat
    for i in range(shift_nr):
        for j in range(len(production_columns)):
            groups[group][production_columns[j]+'_PER_MOMENT_'+str(i+1)]=groups[group][production_columns[j]].shift(periods=-(i+1), fill_value=0)
    '''
    if gr_nr==1:
        gen_data3=pd.DataFrame(columns =groups[group].keys())
    
    gen_data3=pd.concat([gen_data3,groups[group]], ignore_index=True)
    
gen_data3

,DATE_TIME,SOURCE_KEY,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,...,IRRADIATION_PER_INVERTER_MOMENT_SHIFT_MINUS_3,IRRADIATION_PER_INVERTER_DAY_SHIFT_MINUS_3,DC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,AC_DIF_AVG_PER_INVERTER_WHOLE_PERIOD,DC_DIF_AVG_PER_INVERTER_PER_DAY,AC_DIF_AVG_PER_INVERTER_PER_DAY,DC_AVG_DIF_PER_DAY,AC_AVG_DIF_PER_DAY,DC_DIF_AVG_PER_MOMENT,AC_DIF_AVG_PER_MOMENT
0,2020-06-07 00:00:00,1BY6WEcLGh8j5v7,22.055740,20.297180,0.0,4135001.0,0.0,0.0,0.000,6418482.0,...,0.0,0.0,-2578.165755,-252.302118,-2262.684524,-221.328925,-3255.892122,-318.247867,0.0,0.0
1,2020-06-07 00:00:00,1IF53ai7Xc0U56Y,22.055740,20.297180,0.0,4135001.0,0.0,0.0,0.000,6355099.0,...,0.0,0.0,-3044.040819,-297.742623,-3510.206845,-343.059747,-3255.892122,-318.247867,0.0,0.0
2,2020-06-07 00:00:00,3PZuoBAID5Wc2HD,22.055740,20.297180,0.0,4135001.0,0.0,0.0,812.000,7158982.0,...,0.0,0.0,-3038.251607,-297.159468,-3528.917039,-344.862630,-3255.892122,-318.247867,0.0,0.0
3,2020-06-07 00:00:00,7JYdWkrLSPkdwr4,22.055740,20.297180,0.0,4135001.0,0.0,0.0,789.625,7770457.0,...,0.0,0.0,-2963.312661,-289.870968,-3433.260789,-335.559598,-3255.892122,-318.247867,0.0,0.0
4,2020-06-07 00:00:00,McdE0feGgRqW7Ca,22.055740,20.297180,0.0,4135001.0,0.0,0.0,0.000,7331160.0,...,0.0,0.0,-2996.959925,-293.136026,-2995.109003,-292.796131,-3255.892122,-318.247867,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23227,2020-06-17 23:45:00,uHbuxQJl8lW7ozc,21.909288,20.427972,0.0,4135001.0,0.0,0.0,5967.000,7287002.0,...,0.0,0.0,-3021.100196,-295.486773,-2556.190960,-250.116879,-2517.888173,-246.393720,0.0,0.0
23228,2020-06-17 23:45:00,wCURE6d3bPkepu2,21.909288,20.427972,0.0,4135001.0,0.0,0.0,5147.625,7028601.0,...,0.0,0.0,-2933.156765,-286.922068,-2516.066071,-246.225078,-2517.888173,-246.393720,0.0,0.0
23229,2020-06-17 23:45:00,z9Y9gH1T5YWrNuG,21.909288,20.427972,0.0,4135001.0,0.0,0.0,5819.000,7251204.0,...,0.0,0.0,-2874.811372,-281.236710,-2477.912277,-242.506961,-2517.888173,-246.393720,0.0,0.0
23230,2020-06-17 23:45:00,zBIq5rxdHJRwDNY,21.909288,20.427972,0.0,4135001.0,0.0,0.0,5817.000,6583369.0,...,0.0,0.0,-2978.415195,-291.326738,-2488.603646,-243.529178,-2517.888173,-246.393720,0.0,0.0


#other labels adding
'''
* label "INCORRECT_total_yield" - yield per period decreases
* label "INCORRECT_according_to_DC_vs_AC" - DC and AC ratio is off
* label "INCORRECT_according_to_DC_AC_DAILY_YIELD" - DC and AC increases, while DAILY_YIELD does not
'''

In [16]:
#Test andmestik (drop first three train days)
gen_data3=gen_data3[6336:]

#Train andmestik - nill

In [17]:
gen_data3.to_csv(path_or_buf='Data/Test_Plant1_added_features.csv')

In [18]:
#katse=pd.read_csv("Data/Test_Plant1_added_features.csv")

In [19]:
#katse